In [7]:
'''
Extract the data from an xml file previously downloaded from : 
https://archive.org/download/stackexchange
 '''
# import of librairy

import xml.etree.ElementTree as ET
import pymongo
import html2markdown
from lxml import etree
from collections import defaultdict

# connection to mongodb
client = pymongo.MongoClient("mongodb://localhost:27017/")


#Building a mongo collection from an xml file. 
#For each element, we keep only the following informations: 
# id , PostTypeId , AcceptedAnswerId , Body

def extractPosts(mycol,file):
    
    tree = ET.parse(file)
    root = tree.getroot()
   
    import time
    
    start_time = time.time()
    
    liste_des_reponsesId=[]
    for child in root:
        type_id = child.attrib['PostTypeId']

        try:
            rep_id = child.attrib['AcceptedAnswerId']
        except:
            rep_id = 0

        if (type_id=='1' and rep_id !=0) : 
            quest = child.attrib['Body']
            question = html2markdown.convert(quest) 
            liste_des_reponsesId.append(rep_id)
            dict = [{'_id':child.attrib['Id'], 'PostTypeId':1, 'AcceptedAnswerId':rep_id, 'question':question}]
            x = mycol.insert(dict)

    for index in liste_des_reponsesId : 
        found = False
        for child in root:
            if index == child.attrib['Id'] :
                rep = child.attrib['Body']
                reponse = html2markdown.convert(rep)
                dict = [{'_id':index , 'PostTypeId':2,  'response':reponse}]
                x = mycol.insert(dict)
                found = True
            if found : break

    end_time = time.time()
    T = end_time - start_time
    return T, mycol.count()


mydb = client["NougatBD2"]

# creation of the collection astro2
mycol = mydb["astro2"]

file = "/home/sacia/Documents/WorkSpace/briefAgile/astronomy.stackexchange.com/Posts.xml"
time, count = extractPosts(mycol,file)


/home/sacia/anaconda3/envs/MachineLearning/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/home/sacia/anaconda3/envs/MachineLearning/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/home/sacia/anaconda3/envs/MachineLearning/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


In [9]:
print (f" Time : {round(time,3)}    count : {count}")

 Time : 53.167    count : 10184


In [ ]:
# creation of the collection 'bioinfo' 
mycol = mydb["bioinfo"]
file = "/home/sacia/Documents/WorkSpace/briefAgile/bioinformtics.stackexchange.com/Posts.xml"
time, count = extractPosts(mycol,file)

# # creation of the collection 'datascience
mycol = mydb["datascience"]
file = "/home/sacia/Documents/WorkSpace/briefAgile/datascience.stackexchange.com/Posts.xml"
time, count = extractPosts(mycol,file)

In [12]:
# display of collections created
mydb.list_collection_names()

['astro2', 'bioinfo', 'datascience', 'Feedback']

In [ ]:
# Create index for 3 collections : 'astro2', 'bioinfo', 'datascience
mydb.astro2.create_index( [('question','text')],background = True)


In [18]:
mydb.astro2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'NougatBD2.astro2'},
 'question_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'NougatBD2.astro2',
  'weights': SON([('question', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [22]:
# Get indexes
mycol.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'NougatBD2.astro2'},
 'question_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'NougatBD2.astro2',
  'weights': SON([('question', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [25]:
mydb.bioinfo.create_index( [('question','text')],background = True)

'question_text'

In [27]:
# Get indexes
mydb.bioinfo.index_information()


{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'NougatBD2.bioinfo'},
 'question_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'NougatBD2.bioinfo',
  'background': True,
  'weights': SON([('question', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [28]:
mydb.datascience.create_index( [('question','text')],background = True)
# Get indexes
mydb.datascience.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'NougatBD2.datascience'},
 'question_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'NougatBD2.datascience',
  'background': True,
  'weights': SON([('question', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [30]:
# the function resarchQuestion takes a question as input and sends us 
# AcceptedAnswerId of the answer

def resarchQuestion(question):
    mydoc = mycol.find({"$text": {"$search": question}},{"score": {"$meta": "textScore"}})
    mydoc1= mydoc.sort([("score", {"$meta":"textScore"})]).limit(1)
    resp =[]
    for x in mydoc1:
        print("id question : ", x['_id'])
        print("id score : ", x['score'])
        id = x['AcceptedAnswerId']
        return id

question = "how to calculte the inclination of a satellite"
id = resarchQuestion(question)
print(f"acceptedId : {id}")


id question :  12228
id score :  1.658
acceptedId : 22262


In [31]:
#get_response thisfunction that takes the id as input and returns the corresponding response

def get_response(_id) : 
    myquery1 = { "_id": _id }
    reponses = mycol.find(myquery1)
    for rep in reponses :
        reponse = rep['response']
        print(_id)
    return reponse

print(get_response(id))

22262
The most simple Lidov-Kozai model is that of a massless object ($m\_1$ in your diagram) rotating a massive object ($m\_0$), which is itself in an orbit with another massive object ($m\_2$).

This is a hierarchical 3-body system ($m\_2$ is assumed to always be far enough from $m\_0$ and $m\_1$). It is easier to look at as two 2-body orbits:

Inner orbit - $m\_0$ and $m\_1$

Outer orbit - $m\_2$ and $m\_1$+$m\_0$

Since $m\_1$ is massless, the outer orbit is not affected by it and is a simple Kepler orbit of 2 bodies ($m\_0$ and $m\_2$) with fixed parameters. Because of that, the outer orbit defines the coordinates system and lie in XY plane, with its angular momentum $\\vec{L}\_{out}=L\_{out}\\hat{z}$.What we really have here is a motion of a test-particle ($m\_1$ a.k.a the perturbed) around a massive object ($m\_0$) in a binary system (with $m\_2$). The inner orbit can be viewed as a Kepler orbit with a perturbation due to $m\_2$ (a.k.a the perturber). Its parameters do change wi